<a href="https://colab.research.google.com/github/ian293382/Python_Pandas_Numpy/blob/main/CurrencySqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#練習：台灣銀行牌告利率提供了每日的外幣匯率價格，包含美金、港幣、英鎊等；請試著撰寫爬蟲程式將台灣銀行牌告利率的資訊抓下來，並且整理成 Pandas 中的 DataFrame 格式。

# ❏ Sample Input：https://rate.bot.com.tw/xrt?Lang=zh-TW（資料來源：台灣銀行牌告利率 ）

# ❏ Sample Output：將外幣對台幣匯率整理成 DataFrame

# ❏ Sample Code：

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'

res = requests.get(url)

first_response = BeautifulSoup(res.text, features="html.parser")

# 抓日期
date_class = first_response.find('p', class_='text-info')
date = date_class.find('span', class_='time').text.strip().split(' ')[0]
print(date)


# 抓到表格了
# table = first_response.find_all('table', class_='table')
table = first_response.find('table')

# table.tr 可以直接拆掉外層的Table
# print(filter)

currency_elements = table.find_all('div',class_='xrt-cur-indent')

# print(table.find_all('div',class_='xrt-cur-indent'))

# 卡在這邊 如何把資料放進去array
currency_list = [div.text.strip() for div in currency_elements]

# table.find_all('tr' , 'data-table'=='本行現金賣出' ) 這樣可以抓到第二 column 資料 誤解
#  [data.text.strip() for data in a] 之後會有一堆 \n \p 代表你沒拆好裡面還有表格資料 不是最底層


# sale_out_elements = table.find_all('td' , class_="rate-content-cash")
# 如果你使用的 text.strip()之後有
# 這樣幹會出現 0 2 4 = 買入 , 1 3 5 = 賣出
# sale_out_elements = table.find_all('td' , {'data-table':"本行現金賣出"}) 會抓到兩籃數據 確定有分別 現金匯率 即期匯率就不用額外寫

sale_cash_elements = table.find_all('td' , { 'data-table':"本行現金賣出", 'class':'display_none_print_show' })
sale_cash_list = [data.text.strip() for data in sale_cash_elements]

buy_cash_elements = table.find_all('td', { 'data-table': '本行現金買入', 'class':'display_none_print_show' })
buy_cash_list = [data.text.strip() for data in buy_cash_elements]

buy_spot_elements = table.find_all('td', { 'data-table': '本行即期買入', 'class':'display_none_print_show' })
buy_spot_list = [data.text.strip() for data in buy_spot_elements]

sale_spot_elements = table.find_all('td', { 'data-table': '本行即期賣出', 'class':'display_none_print_show' })
sale_spot_list = [data.text.strip() for data in sale_spot_elements]

result = pd.DataFrame({
    '幣別 currency': currency_list,
    '現金本行買入': buy_cash_list,
    '現金本行賣出 ': sale_cash_list,
    '即期本行買入': buy_spot_list,
    '即期本行賣出': sale_spot_list,
    '時間': date
})

print(result)



2024/05/28
   幣別 currency   現金本行買入  現金本行賣出   即期本行買入  即期本行賣出          時間
0     美金 (USD)    31.78    32.45  32.105  32.255  2024/05/28
1     港幣 (HKD)    3.966     4.17   4.087   4.157  2024/05/28
2     英鎊 (GBP)    39.92    42.04  40.815  41.445  2024/05/28
3     澳幣 (AUD)    21.04    21.82  21.255    21.6  2024/05/28
4   加拿大幣 (CAD)    23.11    24.02   23.44   23.77  2024/05/28
5   新加坡幣 (SGD)    23.28    24.19   23.75   23.97  2024/05/28
6   瑞士法郎 (CHF)    34.43    35.63   35.04   35.43  2024/05/28
7     日圓 (JPY)    0.196   0.2088  0.2028  0.2078  2024/05/28
8    南非幣 (ZAR)        -        -    1.71     1.8  2024/05/28
9    瑞典幣 (SEK)     2.65     3.17    2.98     3.1  2024/05/28
10    紐元 (NZD)    19.31    20.16   19.64   19.94  2024/05/28
11    泰幣 (THB)   0.7525   0.9425  0.8619  0.9079  2024/05/28
12  菲國比索 (PHP)   0.4872   0.6192       -       -  2024/05/28
13   印尼幣 (IDR)  0.00168  0.00238       -       -  2024/05/28
14    歐元 (EUR)    34.13    35.47  34.645  35.245  2024/05/28
15    韓元 (KRW

In [18]:
import sqlite3

conn = sqlite3.connect('test.db')
conn.execute('''CREATE TABLE IF NOT EXISTS currencyrates
             (id INTEGER PRIMARY KEY,
              currency TEXT NOT NULL,
              buy_cash_rate  DECIMAL(10,5),
              sale_cash_rate DECIMAL(10,5),
              buy_spot_list DECIMAL(10,5),
              sale_spot_list DECIMAL(10,5),
              date DATE
             );''')
# 創建指令
conn.commit()
# 將 DataFrame 插入到 SQLite 表中
result.to_sql('currency_rates', conn, if_exists='append', index=False)

query = "SELECT * FROM currency_rates"
df = pd.read_sql_query(query, conn)

print(df)
conn.close()

   幣別 currency   現金本行買入  現金本行賣出   即期本行買入  即期本行賣出          時間
0     美金 (USD)    31.78    32.45  32.105  32.255  2024/05/28
1     港幣 (HKD)    3.966     4.17   4.087   4.157  2024/05/28
2     英鎊 (GBP)    39.92    42.04  40.815  41.445  2024/05/28
3     澳幣 (AUD)    21.04    21.82  21.255    21.6  2024/05/28
4   加拿大幣 (CAD)    23.11    24.02   23.44   23.77  2024/05/28
..         ...      ...      ...     ...     ...         ...
71    歐元 (EUR)    34.13    35.47  34.645  35.245  2024/05/28
72    韓元 (KRW)  0.02189  0.02579       -       -  2024/05/28
73   越南盾 (VND)  0.00102  0.00143       -       -  2024/05/28
74   馬來幣 (MYR)    5.809    7.329       -       -  2024/05/28
75   人民幣 (CNY)    4.338      4.5   4.405   4.465  2024/05/28

[76 rows x 6 columns]
